In [1]:
import glob
import zipfile
import json
import pandas as pd
import pickle
from pprint import pprint

In [2]:
data_path = "/home/singh_shruti/data/mag_aminer/"

In [3]:
paper_files = glob.glob(data_path + "mag_papers_[0-9].zip")
author_files = glob.glob(data_path + "mag_authors_[0-9].zip")

In [ ]:
mag_papers = {}

for f in paper_files:
    archive = zipfile.ZipFile(f, 'r')
    zip_files = archive.namelist()
    print(zip_files)
    for fi in zip_files:
        bytes_text = archive.read(fi)
        print("Read file {} in archive {}".format(fi, zip_files))
        try:
            str_list = bytes_text.decode('utf-8').split("\n")
            print("Length of file: ", len(str_list))
            idx = 0
            
            for l in str_list:
                if idx % 1000000 == 0:
                    print(idx)
                idx += 1
                try:
                    paper_dict = json.loads(l)
                    if "year" in paper_dict and paper_dict["year"]:
                        if paper_dict["year"] in [2016, 2017, 2018]:
                            if "venue" in paper_dict and paper_dict["venue"] and "id" in paper_dict["venue"]:
                                if paper_dict["venue"]["id"] == "2584161585":
                                    if "title" in paper_dict:
                                        title = paper_dict["title"].lower()
                                        title = ''.join(filter(str.isalpha, title))
                                        mag_papers[title] = paper_dict
                                else:
                                    if "title" in paper_dict:
                                        title = paper_dict["title"].lower()
                                        title = ''.join(filter(str.isalpha, title))
                                        mag_papers[title] = paper_dict
                            elif "title" in paper_dict:
                                title = paper_dict["title"].lower()
                                title = ''.join(filter(str.isalpha, title))
                                mag_papers[title] = paper_dict
                    else:
                        if "venue" in paper_dict and paper_dict["venue"] and "id" in paper_dict["venue"]:
                            if paper_dict["venue"]["id"] == "2584161585":
                                if "title" in paper_dict:
                                    title = paper_dict["title"].lower()
                                    title = ''.join(filter(str.isalpha, title))
                                    mag_papers[title] = paper_dict
                            else:
                                if "title" in paper_dict:
                                    title = paper_dict["title"].lower()
                                    title = ''.join(filter(str.isalpha, title))
                                    mag_papers[title] = paper_dict
                        elif "title" in paper_dict:
                            title = paper_dict["title"].lower()
                            title = ''.join(filter(str.isalpha, title))
                            mag_papers[title] = paper_dict
                except Exception as ex:
                    print("err for json loading")
                    print(ex)
        except Exception as ex:
            print("err for bytes decoding")
            print(ex)